In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
orientados = pd.read_excel(r'C:\Users\jober\Downloads\Indicadores_orientacion_SISE_2024-2025.xlsx', sheet_name='BD_Indicador_1')
registrados = pd.read_excel(r'C:\Users\jober\Downloads\BD_Registro_2024_2025_acumulado.xlsx',sheet_name='BD_Acumulado-2024-2025')
psicologas = pd.read_excel(r'C:\Users\jober\Downloads\Registro de Orientación Ocupacional a Buscadores de Empleo 2025.xlsx', sheet_name='Orientados')
talleres = pd.read_excel(r'C:\Users\jober\Downloads\Indicadores_orientacion_SISE_2024-2025.xlsx', sheet_name='Reporte_Indicador_2')

c:\Users\jober\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
orientados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2843 entries, 0 to 2842
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Indicador                  2843 non-null   object        
 1   TipoDireccionamiento       2843 non-null   object        
 2   TipoDocumento              2843 non-null   object        
 3   NumeroDocumento            2843 non-null   int64         
 4   CorreoElectronico          2766 non-null   object        
 5   PrimerNombre               2843 non-null   object        
 6   SegundoNombre              2504 non-null   object        
 7   PrimerApellido             2843 non-null   object        
 8   SegundoApellido            2803 non-null   object        
 9   Sexo                       2843 non-null   object        
 10  Ciudad                     2843 non-null   object        
 11  Departamento               2839 non-null   object        
 12  Area  

In [5]:
orientados.columns = orientados.columns.str.lower()
talleres.columns = talleres.columns.str.lower()

In [6]:
for column in ['fechaagendamiento', 'fechaejecucion', 'fechaevaluacion']:
    orientados[column] = pd.to_datetime(orientados[column])
    talleres[column] = pd.to_datetime(talleres[column])

In [7]:
for name in orientados.columns:
    orientados[name] = orientados[name].apply(lambda x: x.lower() if isinstance(x, str) else x)
    talleres[name] = talleres[name].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [8]:
orientados = orientados.rename(columns={
    'fechaagendamiento':'fechaagendamiento_orientacion',
    'fechaejecucion':'fechaejecucion_orientacion',
    'fechaevaluacion':'fechaevaluacion_orientacion',
    'usuarionombre':'orientador',
})
talleres = talleres.rename(columns={
    'fechaagendamiento':'fechaagendamiento_taller',
    'fechaejecucion':'fechaejecucion_taller',
    'fechaevaluacion':'fechaevaluacion_taller',
    'usuarionombre':'tallerista',
})

In [9]:
orientados['mes_orientado'] = orientados['fechaejecucion_orientacion'].dt.month
orientados['mes_orientado'] = pd.to_numeric(orientados['mes_orientado'], errors='coerce').round().astype('Int64')

orientados['año_orientado'] = orientados['fechaejecucion_orientacion'].dt.year
orientados['año_orientado'] = pd.to_numeric(orientados['año_orientado'], errors='coerce').round().astype('Int64')

talleres['mes_taller'] = talleres['fechaejecucion_taller'].dt.month
talleres['mes_taller'] = pd.to_numeric(talleres['mes_taller'], errors='coerce').round().astype('Int64')

talleres['año_taller'] = talleres['fechaejecucion_taller'].dt.year
talleres['año_taller'] = pd.to_numeric(talleres['año_taller'], errors='coerce').round().astype('Int64')

In [10]:
orientados[(orientados['mes_orientado'] == 3)&(orientados['año_orientado'] == 2025)]['sexo'].value_counts()

sexo
f    108
m     71
Name: count, dtype: int64

In [11]:
registrados.columns = registrados.columns.str.replace('Número Documento','numerodocumento')
psicologas.columns = psicologas.columns.str.replace('NUMERO.1','numerodocumento')
psicologas = psicologas.drop('Unnamed: 22', axis=1)

In [12]:
for name in psicologas.columns:
    psicologas[name] = psicologas[name].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [13]:
talleres = talleres.groupby('numerodocumento').agg({
    'indicador': 'first',
    'tipodireccionamiento': 'first', 
    'tipodocumento': 'first', 
    'correoelectronico': 'first', 
    'primernombre': 'first', 
    'segundonombre': 'first', 
    'primerapellido': 'first',
    'segundoapellido': 'first', 
    'sexo': 'first', 
    'ciudad': 'first', 
    'departamento': 'first', 
    'area': 'first', 
    'tipo': 'first',
    'subtipo': 'first', 
    'nombreportafolio': 'first', 
    'nombreconvocatoria': 'first',
    'fechaagendamiento_taller': 'first', 
    'fechaejecucion_taller': 'first',
    'fechaevaluacion_taller': 'first', 
    'aprobacion': 'first', 
    'porcentajeasistencia': 'first',
    'prestadornombre': 'first', 
    'institucionnombre': 'first',
    'instituciondireccion': 'first',
    'institucionmunicipio': 'first', 
    'instituciondepartamento': 'first',
    'programagobiernosino': 'first', 
    'programagobierno': lambda x: ','.join(map(str, x.dropna().unique())), 
    'alianzasentidadesexternas': 'first',
    'tallerista': 'first', 
    'agencianombre': 'first', 
    'numerotelefono': 'first',
    'mes_taller': 'first',
    'año_taller': 'first'
}).reset_index()

In [14]:
orientados = orientados.merge(talleres, on='numerodocumento',how='outer')

In [15]:
columns_to_merge = ['indicador', 'tipodireccionamiento', 'tipodocumento',
       'correoelectronico', 'primernombre','segundonombre', 'primerapellido', 
       'segundoapellido', 'sexo','ciudad', 'departamento', 'area', 'tipo', 'subtipo',
       'nombreportafolio', 'nombreconvocatoria', 'aprobacion',
       'porcentajeasistencia', 'prestadornombre', 'institucionnombre',
       'instituciondireccion', 'institucionmunicipio',
       'instituciondepartamento', 'programagobiernosino',
       'programagobierno', 'alianzasentidadesexternas', 
       'agencianombre', 'numerotelefono']

for col in columns_to_merge:
    orientados[col] = orientados[f'{col}_x'].combine_first(orientados[f'{col}_y'])
    orientados.drop(columns=[f'{col}_x', f'{col}_y'], inplace=True)

In [16]:
registrados = registrados.groupby('numerodocumento').agg({
    'No. ': 'first',
    'Programa / Aliado\n(Si aplica)': 'first',
    'Barrio donde vive': 'first',
    'Tipo Documento': 'first',
    'TIPO_REGISTRO':'first',
    'Nombres': 'first',
    'Apellidos': 'first',
    'Celular': 'first',
    'Teléfono': 'first',
    'Canal de Registro': 'first',
    'Edad':'first',
    'Rango_Edad': 'first',
    'Género': 'first',
    'Nivel de Estudio': 'first',
    'Título Homologado':'first',
    'Ciudad de Residencia':'first', 
    'Email':'first',
    'Fecha Registro' :'first',
    'Programa de Gobierno': lambda x: ','.join(map(str, x.dropna().unique())),
    'Condiciones Especiales': lambda x: ','.join(map(str, x.dropna().unique())),
    'Detalle Discapacidades':'first', 
    'Situación Laboral':'first', 
    'Agente Registra':'first',
    'Fecha Actualización':'first',
    '% Hoja Vida':'first',
    'Prestador Anterior':'first',
    'Fecha Cambio Prestador':'first', 
    'Vereda/Localidad/Centro Poblado':'first',
    'Pertenece A':'first', 
    'SISE_OFFLINE':'first', 
    'Mes':'first',
    'Año':'first', 
    'Punto Atención':'first',
}).reset_index()

In [17]:
psicologas = psicologas.groupby('numerodocumento').agg({
    'MES': 'first', 
    'NUMERO': 'first', 
    'FECHA': 'first', 
    'ORIENTADOR': 'first', 
    'NOMBRE': 'first', 
    'TD': 'first',
    'GENERO': 'first', 
    'EDAD': 'first', 
    'RANGO': 'first', 
    'TELEFONO': 'first', 
    'BARRIO': 'first',
    'NIVEL DE FORMACIÓN': 'first', 
    'FORMACIÓN': 'first', 
    'EXPERIENCIA LABORAL': 'first',
    'POBLACIÓN': lambda x: ','.join(map(str, x.dropna().unique())),
    'CORREO ELECTRONICO': 'first', 
    'TALLER FIS': lambda x: ','.join(map(str, x.dropna().unique())),
    'OBSERVACIONES': 'first',
    'INTÉRES CURSO FORMACIÓN': 'first', 
    'VALIDACIÓN DE BACHILLERATO (SI / NO)': 'first',
    'A TENER EN CUENTA': 'first'
}).reset_index()

In [18]:
orientados = orientados.merge(registrados[['numerodocumento', 'Programa de Gobierno','Condiciones Especiales']], on='numerodocumento', how='left')

In [19]:
orientados = orientados.merge(psicologas[['numerodocumento','EDAD','POBLACIÓN']], on='numerodocumento', how='left')

In [20]:
orientados['Condiciones Especiales'] = orientados.apply(
    lambda row: f"{row['Condiciones Especiales']}, {row['POBLACIÓN']}" if pd.notnull(row['POBLACIÓN']) else row['Condiciones Especiales'],
    axis=1
)

In [21]:
orientados = orientados.drop('POBLACIÓN', axis=1)

In [22]:
orientados.columns = orientados.columns.str.lower()
orientados.columns = orientados.columns.str.replace(' ','_')

In [23]:
orientados['edad'] = pd.to_numeric(orientados['edad'], errors='coerce').round().astype('Int64')

In [24]:
# Cleaning column 'edad': 
def range_age(age):
    if  0 < age < 18:
        return '< 18'
    elif 18 <= age < 29:
        return '18-28'
    elif 29 <= age < 40:
        return '29-39'
    elif 40 <= age < 50:
        return '40-49'
    elif 50 <= age < 60:
        return '50-59'
    elif 60 <= age < 2000:
        return '> 60'
    else:
        return np.nan

orientados['rango_de_edad'] = orientados['edad'].apply(range_age)

In [25]:
# orientados.head()

# GRUPOS ETNICOS

In [26]:
# Ensure conditions are strings and lowercase
orientados['condiciones_especiales'] = orientados['condiciones_especiales'].astype(str).str.lower().fillna('')

# Define filters
filter_afro = orientados['condiciones_especiales'].str.contains('negr|afro|mulat|palen', regex=True)
filter_raizal = orientados['condiciones_especiales'].str.contains('raiz', regex=True)
filter_indig = orientados['condiciones_especiales'].str.contains('indí', regex=True)
filter_git = orientados['condiciones_especiales'].str.contains('git', regex=True)

def get_ethnic_groups(row):
    groups = []
    if filter_afro[row.name]: groups.append("Afrodescendiente")
    if filter_raizal[row.name]: groups.append("Raizal y/o Isleño")
    if filter_indig[row.name]: groups.append("Indígenas")
    if filter_git[row.name]: groups.append("Gitano")
    return groups if groups else np.nan  # Use NaN for empty lists

# Apply the function
orientados["grupos_etnicos"] = orientados.apply(get_ethnic_groups, axis=1)

# VICTIMAS DEL CONFLICTO ARMADO

In [27]:
# Poblacion VCA 
orientados['programa_de_gobierno'].astype(str)
orientados['programa_de_gobierno'] = orientados['programa_de_gobierno'].fillna('')

orientados['vca'] = pd.Series(dtype='object') # Adding a new column

filter_vca = (
    (orientados['programa_de_gobierno'].str.contains('armado', na=False)) |
    (orientados['condiciones_especiales'].str.contains('vca|v.c.a'))
)

orientados.loc[filter_vca,'vca'] = 'VCA'

In [28]:
orientados['condiciones_especiales'].unique()

array(['ninguna de las anteriores, ningún grupo étnico,retornado o migrante, migrante',
       'nan',
       'migración colombia (jornada especial 7,8,9-04-2025),ninguna, migrante',
       'nan, migrante', ', migrante',
       'ninguna de las anteriores, ningún grupo étnico', 'nan, ninguna',
       'migrante, ninguna', ', migrante-mch', ', jovenes',
       'ninguna, migrante', 'ninguna, jovenes', 'nan, jovenes', ', mch',
       'nan, migrnte', 'retornado o migrante, migrante',
       'ninguna de las anteriores, ningún grupo étnico,ninguna,migrante',
       'retornado, jovenes', 'migrante, migrante', 'migrante, jovenes',
       ', ninguna', 'retornado, migrante',
       'ninguna de las anteriores, ningún grupo étnico,retornado o migrante',
       'nan, vca', 'ninguna, ninguna', 'migrante, migrante venezolano',
       '', 'retornado o migrante, ninguna',
       'nan, condicion de discapacidad fisica',
       'ninguna de las anteriores, ningún grupo étnico,ninguna, migrante',
       'ning

# PERSONAS EN CONDICION DE DISCAPACIDAD

In [29]:
# Initialize the column with NaN directly
orientados['discapacidad'] = pd.Series([np.nan] * len(orientados), dtype='object')

# Mapping of patterns to labels
discapacidad_patterns = {
    r'capacidad': 'Discapacidad',
    r'ognitiv|telect': 'Cognitiva o Intelectual',
    r'[ií]sic': 'Física',
    r'visual': 'Visual',
    r'auditiva': 'Auditiva',
    r'múltiple': 'Múltiple',
    r'sordoceguera': 'Sordoceguera',
    r'psicosocial': 'Psicosocial'
}

# Apply the patterns
for pattern, label in discapacidad_patterns.items():
    mask = orientados['condiciones_especiales'].str.contains(pattern, case=False, na=False)
    orientados.loc[mask, 'discapacidad'] = label

# MIGRANTES

In [30]:
# Migrant and internally displaced people
orientados['migrante'] = ""

# Filters applied
filter_mig = (
    (orientados['condiciones_especiales'].str.contains('migr|retor')) |
    (orientados['tipodocumento'].str.contains('dni|ppt|ce'))
)

orientados.loc[filter_mig,"migrante"] = "Migrante o Retornado"

# Replace empty strings with NaN
orientados['migrante'] = orientados['migrante'].replace("", np.nan)

# VÍCTIMA DE VIOLENCIA

In [31]:
# Population VVG
orientados['vvg'] = ""

# Filters applied
filter_vvg = (
    (orientados['condiciones_especiales'].str.contains('viole')) |
    (orientados['condiciones_especiales'].str.contains('vvg'))     
)

orientados.loc[filter_vvg,"vvg"] = "vvg"

# Replace empty strings with NaN
orientados['vvg'] = orientados['vvg'].replace("",  np.nan)

# REINSERTADOS

In [32]:
# Reincorporados
orientados['reincorporados'] = ""

# Filters applied
filter_rei = (
    (orientados['condiciones_especiales'].str.contains('rein'))
)

orientados.loc[filter_rei,"reincorporados"] = "reincorporados"

# Replace empty strings with NaN
orientados['reincorporados'] = orientados['reincorporados'].replace("", np.nan)

In [33]:
orientados.head()

,numerodocumento,fechaagendamiento_orientacion,fechaejecucion_orientacion,fechaevaluacion_orientacion,orientador,mes_orientado,año_orientado,fechaagendamiento_taller,fechaejecucion_taller,fechaevaluacion_taller,tallerista,mes_taller,año_taller,indicador,tipodireccionamiento,tipodocumento,correoelectronico,primernombre,segundonombre,primerapellido,segundoapellido,sexo,ciudad,departamento,area,tipo,subtipo,nombreportafolio,nombreconvocatoria,aprobacion,porcentajeasistencia,prestadornombre,institucionnombre,instituciondireccion,institucionmunicipio,instituciondepartamento,programagobiernosino,programagobierno,alianzasentidadesexternas,agencianombre,numerotelefono,programa_de_gobierno,condiciones_especiales,edad,rango_de_edad,grupos_etnicos,vca,discapacidad,migrante,vvg,reincorporados
0,511732,2025-01-29,2025-01-29,2025-01-29,ileana margarita palma restrepo,1,2025,2025-01-29,2025-01-29,2025-01-29,ileana margarita palma restrepo,1,2025,indicador 1,registro continuo,dni,felix_aragon@outlook.com,felix,javier,aragón,conde,m,barranquilla,atlántico,orientación laboral,entrevista,grupal,entrevista orientación laboral san josé,fis 29 enero 2025 presencial baq bilingue,si,100.0,NaN,NaN,NaN,NaN,NaN,no,,NaN,alcaldia distrital de barranquilla - barranqui...,3116545755.0,,"ninguna de las anteriores, ningún grupo étnico...",50,50-59,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
1,669436,NaT,NaT,NaT,NaN,<NA>,<NA>,2023-08-31,2023-08-31,2024-08-22,mileidys isabel oñoro perez,8,2023,indicador 2,registro continuo,dni,ebenezergomezpino@hotmail.com,nestor,luis,gomez,graterol,m,soledad,atlántico,orientación laboral,talleres,grupal,taller fis,taller fis agosto 31 de 2023 mp,si,80.0,NaN,NaN,NaN,NaN,NaN,no,,NaN,alcaldia distrital de barranquilla - barranqui...,3207865405,,nan,<NA>,NaN,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
2,682244,2025-04-28,2025-04-28,2025-04-28,liliana eugenia arciniegas navarro,4,2025,NaT,NaT,NaT,NaN,<NA>,<NA>,indicador 1,registro continuo,dni,reghaxo25@gmail.com,keinyn,alexander,noguera,noguera,m,soledad,atlántico,orientación laboral,entrevista,individual,entrevista orientación laboral san jose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,alcaldia distrital de barranquilla - barranqui...,3028432132.0,,"migración colombia (jornada especial 7,8,9-04-...",28,18-28,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
3,682244,2025-04-29,2025-04-30,2025-04-30,liliana eugenia arciniegas navarro,4,2025,NaT,NaT,NaT,NaN,<NA>,<NA>,indicador 1,registro continuo,dni,reghaxo25@gmail.com,keinyn,alexander,noguera,noguera,m,soledad,atlántico,orientación laboral,entrevista,individual,entrevista orientación laboral san jose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,alcaldia distrital de barranquilla - barranqui...,3028432132.0,,"migración colombia (jornada especial 7,8,9-04-...",28,18-28,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
4,692862,2024-04-23,2024-04-23,2024-04-23,ileana margarita palma restrepo,4,2024,2024-04-26,2024-04-26,2024-04-26,enrique eduardo escorcia,4,2024,indicador 1,registro continuo,dni,eneisys@gmail.com,eneisy,johanna,soto,coroy,f,barranquilla,atlántico,orientación laboral,entrevista,grupal,entrevista orientación laboral san josé,fis presencial 25 abril 2024 ee,si,100.0,NaN,NaN,NaN,NaN,NaN,no,,NaN,alcaldia distrital de barranquilla - barranqui...,3115774900.0,,"nan, migrante",44,40-49,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN


In [34]:
orientados.shape

(3222, 51)

In [35]:
# La información es:
# pd.crosstab(orientados['rango_de_edad'],orientados['discapacidad'])

# Reportes de población

In [43]:
# Personas orientadas
mes_orientado = 4
año_orientado = 2025
f_orientado = (orientados['mes_orientado'] == mes_orientado) & (orientados['año_orientado'] == año_orientado)

print(f'El número de hombres orientados durante el mes {mes_orientado} es: {orientados[f_orientado & (orientados['sexo']=='m')]['sexo'].count()}')
print(f'El número de mujeres orientadas durante el mes {mes_orientado} es: {orientados[f_orientado & (orientados['sexo']=='f')]['sexo'].count()}')
print()

print(f'El número de PCD orientadas durante el mes {mes_orientado} es: {orientados[f_orientado]['discapacidad'].count()}')
print(f'El número de victimas orientadas durante el mes {mes_orientado} es: {orientados[f_orientado]['vca'].count()}')
print(f'El número de victimas de violencia de género orientadas durante el mes {mes_orientado} es: {orientados[f_orientado]['vvg'].count()}')
print(f'El número de migrantes orientadas durante el mes {mes_orientado} es: {orientados[f_orientado]['migrante'].count()}')
print(f'El número de personas de grupos étnicos orientadas durante el mes {mes_orientado} es: {orientados[f_orientado]['grupos_etnicos'].count()}')
print(f'El número de reincorporados orientados durante el mes {mes_orientado} es: {orientados[f_orientado]['reincorporados'].count()}')
print()

print(f'El número de adultos mayores orientados durante el mes {mes_orientado} es: {orientados[f_orientado & (orientados['edad'] >= 60)]['sexo'].count()}')
print(f'El número de adultos orientados durante el mes {mes_orientado} es: {orientados[f_orientado & (orientados['edad'] >= 29) & (orientados['edad'] < 60)]['sexo'].count()}')
print(f'El número de jovenes orientados durante el mes {mes_orientado} es: {orientados[f_orientado & (orientados['edad'] <= 28)]['sexo'].count()}')


El número de hombres orientados durante el mes 4 es: 95
El número de mujeres orientadas durante el mes 4 es: 140

El número de PCD orientadas durante el mes 4 es: 36
El número de victimas orientadas durante el mes 4 es: 11
El número de victimas de violencia de género orientadas durante el mes 4 es: 0
El número de migrantes orientadas durante el mes 4 es: 29
El número de personas de grupos étnicos orientadas durante el mes 4 es: 9
El número de reincorporados orientados durante el mes 4 es: 0

El número de adultos mayores orientados durante el mes 4 es: 6
El número de adultos orientados durante el mes 4 es: 133
El número de jovenes orientados durante el mes 4 es: 93


In [44]:
# Personas taller FIS
mes_taller = 4
año_taller = 2025
f_taller = (orientados['mes_taller'] == mes_taller) & (orientados['año_taller'] == año_taller)

print(f'El número de hombres on taller FIS durante el mes {mes_taller} es: {orientados[f_taller & (orientados['sexo']=='m')]['sexo'].count()}')
print(f'El número de mujeres on taller FIS durante el mes {mes_taller} es: {orientados[f_taller & (orientados['sexo']=='f')]['sexo'].count()}')
print()

print(f'El número de PCD con taller FIS durante el mes {mes_taller} es: {orientados[f_taller]['discapacidad'].count()}')
print(f'El número de victimas con taller FIS durante el mes {mes_taller} es: {orientados[f_taller]['vca'].count()}')
print(f'El número de victimas de violencia de género con taller FIS durante el mes {mes_taller} es: {orientados[f_taller]['vvg'].count()}')
print(f'El número de migrantes con taller FIS durante el mes {mes_taller} es: {orientados[f_taller]['migrante'].count()}')
print(f'El número de personas de grupos étnicos con taller FIS durante el mes {mes_taller} es: {orientados[f_taller]['grupos_etnicos'].count()}')
print(f'El número de reincorporados con taller FIS durante el mes {mes_taller} es: {orientados[f_taller]['reincorporados'].count()}')
print()

print(f'El número de adultos mayores con taller FIS durante el mes {mes_taller} es: {orientados[f_taller & (orientados['edad'] >= 60)]['sexo'].count()}')
print(f'El número de adultos con taller FIS durante el mes {mes_taller} es: {orientados[f_taller & (orientados['edad'] >= 29) & (orientados['edad'] < 60)]['sexo'].count()}')
print(f'El número de jovenes con taller FIS durante el mes {mes_taller} es: {orientados[f_taller & (orientados['edad'] <= 28)]['sexo'].count()}')

El número de hombres on taller FIS durante el mes 4 es: 47
El número de mujeres on taller FIS durante el mes 4 es: 44

El número de PCD con taller FIS durante el mes 4 es: 21
El número de victimas con taller FIS durante el mes 4 es: 4
El número de victimas de violencia de género con taller FIS durante el mes 4 es: 0
El número de migrantes con taller FIS durante el mes 4 es: 8
El número de personas de grupos étnicos con taller FIS durante el mes 4 es: 4
El número de reincorporados con taller FIS durante el mes 4 es: 0

El número de adultos mayores con taller FIS durante el mes 4 es: 1
El número de adultos con taller FIS durante el mes 4 es: 36
El número de jovenes con taller FIS durante el mes 4 es: 45


In [38]:
# # LISTADO DE PERSONAS CON DISCAPACIDAD ORIENTADOS
# mes_orientado = 3
# año_orientado = 2025
# f_orientado = (orientados['mes_orientado'] == mes_orientado) & (orientados['año_orientado'] == año_orientado) & (~orientados['orientador'].isna())

# orientados[f_orientado & (~orientados['discapacidad'].isna())][['numerodocumento','primernombre', 'primerapellido']]

In [39]:
orientados.head()

,numerodocumento,fechaagendamiento_orientacion,fechaejecucion_orientacion,fechaevaluacion_orientacion,orientador,mes_orientado,año_orientado,fechaagendamiento_taller,fechaejecucion_taller,fechaevaluacion_taller,tallerista,mes_taller,año_taller,indicador,tipodireccionamiento,tipodocumento,correoelectronico,primernombre,segundonombre,primerapellido,segundoapellido,sexo,ciudad,departamento,area,tipo,subtipo,nombreportafolio,nombreconvocatoria,aprobacion,porcentajeasistencia,prestadornombre,institucionnombre,instituciondireccion,institucionmunicipio,instituciondepartamento,programagobiernosino,programagobierno,alianzasentidadesexternas,agencianombre,numerotelefono,programa_de_gobierno,condiciones_especiales,edad,rango_de_edad,grupos_etnicos,vca,discapacidad,migrante,vvg,reincorporados
0,511732,2025-01-29,2025-01-29,2025-01-29,ileana margarita palma restrepo,1,2025,2025-01-29,2025-01-29,2025-01-29,ileana margarita palma restrepo,1,2025,indicador 1,registro continuo,dni,felix_aragon@outlook.com,felix,javier,aragón,conde,m,barranquilla,atlántico,orientación laboral,entrevista,grupal,entrevista orientación laboral san josé,fis 29 enero 2025 presencial baq bilingue,si,100.0,NaN,NaN,NaN,NaN,NaN,no,,NaN,alcaldia distrital de barranquilla - barranqui...,3116545755.0,,"ninguna de las anteriores, ningún grupo étnico...",50,50-59,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
1,669436,NaT,NaT,NaT,NaN,<NA>,<NA>,2023-08-31,2023-08-31,2024-08-22,mileidys isabel oñoro perez,8,2023,indicador 2,registro continuo,dni,ebenezergomezpino@hotmail.com,nestor,luis,gomez,graterol,m,soledad,atlántico,orientación laboral,talleres,grupal,taller fis,taller fis agosto 31 de 2023 mp,si,80.0,NaN,NaN,NaN,NaN,NaN,no,,NaN,alcaldia distrital de barranquilla - barranqui...,3207865405,,nan,<NA>,NaN,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
2,682244,2025-04-28,2025-04-28,2025-04-28,liliana eugenia arciniegas navarro,4,2025,NaT,NaT,NaT,NaN,<NA>,<NA>,indicador 1,registro continuo,dni,reghaxo25@gmail.com,keinyn,alexander,noguera,noguera,m,soledad,atlántico,orientación laboral,entrevista,individual,entrevista orientación laboral san jose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,alcaldia distrital de barranquilla - barranqui...,3028432132.0,,"migración colombia (jornada especial 7,8,9-04-...",28,18-28,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
3,682244,2025-04-29,2025-04-30,2025-04-30,liliana eugenia arciniegas navarro,4,2025,NaT,NaT,NaT,NaN,<NA>,<NA>,indicador 1,registro continuo,dni,reghaxo25@gmail.com,keinyn,alexander,noguera,noguera,m,soledad,atlántico,orientación laboral,entrevista,individual,entrevista orientación laboral san jose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,alcaldia distrital de barranquilla - barranqui...,3028432132.0,,"migración colombia (jornada especial 7,8,9-04-...",28,18-28,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
4,692862,2024-04-23,2024-04-23,2024-04-23,ileana margarita palma restrepo,4,2024,2024-04-26,2024-04-26,2024-04-26,enrique eduardo escorcia,4,2024,indicador 1,registro continuo,dni,eneisys@gmail.com,eneisy,johanna,soto,coroy,f,barranquilla,atlántico,orientación laboral,entrevista,grupal,entrevista orientación laboral san josé,fis presencial 25 abril 2024 ee,si,100.0,NaN,NaN,NaN,NaN,NaN,no,,NaN,alcaldia distrital de barranquilla - barranqui...,3115774900.0,,"nan, migrante",44,40-49,NaN,NaN,NaN,Migrante o Retornado,NaN,NaN
